# Step 1. Cursor Compose - Đóng Dockerfile và docker-compose.yml cho source frontend cũ để deploy port 30001

QDant Free trước để upload Vector Database lên. 
```bash
1 NODE
DISK: 4GiB
RAM: 1GiB
vCPUs: 0.5
```

## Bổ sung1: Update UI cho đẹp
@front-end 
Giữ nguyên mọi cái, đổi phông màu sang màu khác và biến nó thành UI, UX của nhà Apple, Iphone, Ipad

# Step 2: Cursor từ 2 file: format.md và LegalRAG_VECTOR_DATABASE_CREATE_EXCEL_LuatRAG.ipynb => viết thành .py 

git commit -m "Load Excel Data => Qdant embedding - page_content+metadata:(source, question)

- Viết thêm 1 file test_qdant_connection.py (Cursor)
- Loading data from backend_vector_database/dataset/LegalRAG.xlsx...
Creating 85 document objects with metadata...
Initializing embeddings with model: sentence-transformers/paraphrase-multilingual-mpnet-base-v2
Creating vector database in Qdrant collection: legal_rag
Successfully added 85 documents to Qdrant"

# Step 3: Build Backend từ file pipeline của OpenWebUI trước mình build

Chạy fastAPI để test real time ? log để xem bug ý 



To run the FastAPI application for real-time testing and view logs for debugging, I'll provide you with the commands and instructions to do this effectively.

## Running FastAPI for Real-Time Testing

Here are two approaches to run and test your FastAPI application:

### Option 1: Run directly with Python

````bash
# Navigate to your backend directory
cd src/back-end

# Run the FastAPI application with debug logs
python rag_backend.py
````

This will start the server on port 8000 as specified in your code, with auto-reload enabled for development.

### Option 2: Run with Uvicorn directly (more control)

````bash
# Navigate to your backend directory
cd src/back-end

# Run with uvicorn with log level set to debug
uvicorn rag_backend:app --host 0.0.0.0 --port 8000 --reload --log-level debug
````

## Viewing Logs

The application already has print statements for debugging. To enhance logging:

````python:src/back-end/rag_backend.py
# Add this near the top of your file
import logging

# Configure logging
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler()  # Output to console
    ]
)

logger = logging.getLogger("rag-backend")

# Then replace print statements with logger calls
# For example:
# print("Initializing RAG Backend...") becomes:
logger.info("Initializing RAG Backend...")
# print(f"Startup error: {str(e)}") becomes:
logger.error(f"Startup error: {str(e)}")
````

## Testing the API

While the server is running, you can test it using the curl commands from your API_test.md file:

````bash
# Test with local RAG backend (port 8000)
curl -X POST "http://localhost:8000/v1/chat/completions" \
  -H "Content-Type: application/json" \
  -d '{
    "messages": [
      {"role": "user", "content": "Quy tắc giao thông đường bộ là gì?"}
    ],
    "model": "gpt-3.5-turbo"
  }'
````

## Debugging Tips

1. Check the FastAPI interactive docs at http://localhost:8000/docs when running locally
2. Monitor the console output for errors and request processing logs
3. Use the health endpoint to verify service status: `curl http://localhost:8000/health`
4. For Docker container logs: `docker logs legal-rag-backend -f`

These approaches will help you test in real-time and identify any bugs through the logs.


# Step 4: Connect Backend với Frontend vì frontend đang xài cơ chế cũ 







# Phân tích Backend và Frontend

## Backend

Backend đang sử dụng **FastAPI** - một framework Python hiện đại để xây dựng API với hiệu suất cao:

1. **Công nghệ chính**:
   - FastAPI (framework API)
   - Uvicorn (ASGI server)
   - Qdrant (vector database)
   - LangChain (framework RAG)
   - OpenAI API (LLM)

2. **Endpoint chính**:
   - `/v1/chat/completions` - Endpoint chính để xử lý các yêu cầu chat
   - `/health` - Endpoint kiểm tra trạng thái

3. **Đặc điểm**:
   - Cung cấp API tương thích với định dạng OpenAI
   - Chạy trên port 30000
   - Sử dụng Qdrant để tìm kiếm ngữ nghĩa

## Frontend

Frontend đang sử dụng **React** với **Vite** làm build tool:

1. **Công nghệ chính**:
   - React (thư viện UI)
   - Vite (build tool)
   - Tailwind CSS (framework CSS)
   - React Router (quản lý routing)

2. **Cấu trúc**:
   - Có các trang: HomePage, FAQPage, IssuePage
   - Component chính: ChatBot, NavBar

## Phương thức giao tiếp

Dựa trên code hiện tại, **phương thức giao tiếp giữa Frontend và Backend có thể chưa tương thích**:

1. **Backend API**:
   - Cung cấp endpoint `/v1/chat/completions` tương thích với OpenAI API
   - Nhận và trả về dữ liệu theo định dạng OpenAI

2. **Frontend**:
   - Không thấy rõ code gọi API trong các file frontend
   - Component ChatBot.jsx có thể đang sử dụng API khác

## Điều chỉnh cần thiết

Để đảm bảo frontend và backend có thể giao tiếp, cần:

1. **Cập nhật ChatBot.jsx** để gọi đến API backend mới:

```javascript
// Thay đổi từ
const response = await fetch('https://api.openai.com/v1/chat/completions', {
  // ...
});

// Thành
const response = await fetch('http://localhost:30000/v1/chat/completions', {
  // ...
});
```

2. **Cập nhật vite.config.js** để proxy API requests trong môi trường phát triển:

```javascript
export default defineConfig({
  // ...
  server: {
    proxy: {
      '/v1/chat/completions': {
        target: 'http://localhost:30000',
        changeOrigin: true,
      }
    }
  }
});
```

3. **Cập nhật CORS trong backend** để cho phép frontend truy cập:

```python
app.add_middleware(
    CORSMiddleware,
    allow_origins=["http://localhost:5173"],  # Vite dev server port
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)
```

## Kết luận

Backend và Frontend đang sử dụng các công nghệ hiện đại nhưng có thể chưa được cấu hình để giao tiếp với nhau. Cần cập nhật cả hai phía để đảm bảo chúng có thể hoạt động cùng nhau. Phương pháp tốt nhất là cập nhật frontend để gọi API backend mới, hoặc cập nhật backend để tương thích với API mà frontend đang sử dụng.




# Frontend API Integration Report

## Current API Configuration

After examining the frontend code, specifically the `ChatBot.jsx` component, I found that the frontend is currently making API requests to an ngrok endpoint:

```javascript
fetch("https://briefly-knowing-treefrog.ngrok-free.app/rag/" + sourceData + "?q=" + promptInput,
{
  method: "get",
  headers: new Headers({
    "ngrok-skip-browser-warning": "69420",
  }),
})
```

## Issues Identified

1. **Temporary ngrok URL**: The frontend is using a temporary ngrok URL (`briefly-knowing-treefrog.ngrok-free.app`) which will change when the ngrok session restarts.

2. **No connection to Docker backend**: The frontend is not connecting to the Docker containerized backend service we've been configuring at port 30000.

3. **Different API structure**: The current API endpoint structure (`/rag/{source}?q={query}`) doesn't match the OpenAI-compatible endpoint we've implemented in our backend (`/v1/chat/completions`).

## Recommended Fixes

1. **Update API endpoint to use the backend container**:

```javascript
// Replace this:
fetch("https://briefly-knowing-treefrog.ngrok-free.app/rag/" + sourceData + "?q=" + promptInput, ...)

// With this (for local development):
fetch("http://localhost:30000/v1/chat/completions", {
  method: "POST",
  headers: {
    "Content-Type": "application/json",
  },
  body: JSON.stringify({
    messages: [
      {"role": "user", "content": promptInput}
    ],
    model: "gpt-3.5-turbo"
  })
})
```

2. **Update response handling**:
   The current code expects a different response format than what our OpenAI-compatible API returns. We need to update the response handling to match our API's format.

3. **Environment-based configuration**:
   Add environment variables to configure the API URL based on the deployment environment (development, production, etc.).

4. **Docker Compose for frontend and backend**:
   Create a combined Docker Compose setup that includes both the frontend and backend services, with proper networking between them.

These changes will ensure the frontend properly connects to our containerized backend service and handles the responses correctly.


# Step 5: Fix lỗi connect backend với frontend -- mệt muón ngất - 2h-3h để fix lỗi này

# Step 6: Chỉnh UI, UX: 
1. Thêm 1 dark mode
- oke đó, dark mode tác động vào cả bên trong nữa đi 
- Để 3 câu hỏi tham khảo cho lần hỏi đầu tiên, và kéo dài thanh chat ra đi 
---
Thêm Darkmode mé, xịn xò vãi chưởng